In [1]:
# Imports
from Code.h5Constructor import h5Constructor

In [2]:
gc = h5Constructor('./Dataset/CIFs/Train/', './Dataset/h5/Train_GPU/')

Save directory doesn't exist.
Created the save directory at Dataset/h5/Train_GPU


In [3]:
gc.gen_h5s(parallelize=False)


Constructing graphs from cif files:


  0%|          | 0/28 [00:00<?, ?it/s]














  4%|▎         | 1/28 [00:36<16:35, 36.87s/it]


KeyboardInterrupt: 

In [ ]:
from Code.datasetClass import InOrgMatDatasets

test = InOrgMatDatasets('DatasetTest', root='./Dataset/', force_update=False)

Extracting Dataset/DatasetTest/raw/DatasetTest.zip
Processing...
Done!


In [ ]:
test.get(15, data_split='train')

Data(
  x=[12, 4],
  edge_index=[2, 64],
  edge_attr=[88],
  y={
    crystal_type='Flourite',
    atomic_species=[2],
    cell_params=[6],
    np_size=11.59610962646957,
    nd=[2, 580],
    xrd=[2, 580],
    nPDF=[2, 301],
    xPDF=[2, 301],
    sans=[2, 300],
    saxs=[2, 300]
  },
  pos=[12, 3],
  pos_real=[12, 3]
)

In [1]:
import Code.cifCleaning as cc
import pandas as pd

df = cc.cif_cleaning_pipeline('./Dataset/CIFs/COD_subset/', remove_duplicates=True, unwanted_atoms=None)

Cleaning CIFs:   0%|          | 0/42320 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [2]:
df.head()

,filepath,Spacegroup,Element0,Element1,Element2
0,./Dataset/CIFs/ICSD_MonoMetalOxides_cleaned/ic...,167,8,26.0,NaN
1,./Dataset/CIFs/ICSD_MonoMetalOxides_cleaned/ic...,136,8,22.0,NaN
2,./Dataset/CIFs/ICSD_MonoMetalOxides_cleaned/ic...,136,8,22.0,NaN
3,./Dataset/CIFs/ICSD_MonoMetalOxides_cleaned/ic...,40,8,24.0,NaN
4,./Dataset/CIFs/ICSD_MonoMetalOxides_cleaned/ic...,11,8,64.0,NaN


In [3]:
from ase.io import read
import numpy as np

In [4]:
cif1 = read('./Dataset/CIFs/ICSD_MonoMetalOxides_cleaned/icsd_000029_cc.cif')

cif2 = read('./Dataset/CIFs/ICSD_MonoMetalOxides_cleaned/icsd_000096_cc.cif')

/root/anaconda3/envs/dataset_env/lib/python3.7/site-packages/ase/io/cif.py:404: UserWarning: crystal system 'monoclinic' is not interpreted for space group Spacegroup(10, setting=1). This may result in wrong setting!
  setting_name, spacegroup))
/root/anaconda3/envs/dataset_env/lib/python3.7/site-packages/ase/io/cif.py:404: UserWarning: crystal system 'tetragonal' is not interpreted for space group Spacegroup(88, setting=1). This may result in wrong setting!
  setting_name, spacegroup))


In [5]:
np.unique(cif1.get_atomic_numbers())

array([ 8, 41])

In [9]:
any([x in [8, 12] for x in cif1.get_atomic_numbers()])

True

In [13]:
np.array(cif1.get_cell_lengths_and_angles())

/root/anaconda3/envs/dataset_env/lib/python3.7/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)


array([ 21.153 ,   3.8233,  19.356 ,  90.    , 119.8   ,  90.    ])

In [33]:

test_dict = dict(a=[1], b=[2], c=[3])

In [1]:
from pathlib import Path
cif_path = Path('../cif/1/00/00/1000000.cif')

In [8]:
cif_path

PosixPath('../cif/1/00/00/1000000.cif')

In [26]:
import requests, zipfile, io
from pathlib import Path

In [27]:
from itertools import islice, zip_longest

def batched(iterable, n):
    "Batch data into lists of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    it = iter(iterable)
    while True:
        batch = list(islice(it, n))
        if not batch:
            return
        yield batch

In [29]:
api_url = 'https://www.crystallography.net/cod/result?format=lst&nel1=C'
save_folder = './Dataset/CIFs/COD_subset/'
if not Path(save_folder).exists():
    Path(save_folder).mkdir(parents=True)
response = requests.get(api_url)

In [30]:
with open(f'{save_folder}cif_IDs.txt', 'w') as file:
    file.write(response.text)

In [40]:
with open(f'{save_folder}cif_IDs.txt', 'r') as file:
    ids = file.readlines()
    for id_batch in batched(ids, 800):
        requested_ids = ''.join(id_batch)
    
        # Request cif files
        api_url = f'https://www.crystallography.net/cod/result?format=zip&id={requested_ids}'
        response = requests.get(api_url)
        # Extract requested cif files
        zip_file = zipfile.ZipFile(io.BytesIO(response.content))
        zip_file.extractall(save_folder)
        

In [32]:
response.raw

In [6]:
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

BadZipFile: File is not a zip file

In [20]:
zip_file.extractall(save_folder)